In [10]:
import pandas
import numpy
import math


%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
# We will use Keras library
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
# And Scikit-Learn
from keras.models import model_from_yaml
import numpy
import os
import datetime
import time
from time import gmtime, strftime
import os
# Import libraries use for visualization and analysis
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

import cufflinks as cf

from monascaclient import client
from monascaclient import ksclient


from pandas import Series,DataFrame

import math
from __future__ import division

# Import library to execute remote commands for monasca-agent demo
import spur
import matplotlib.pyplot as plt
import seaborn as sns

import time as t
import datetime as dt

In [11]:
KEYSTONE_URL = 'http://157.159.232.218:35357/v3/'
PROJECT_NAME = 'mini-mon'
USERNAME = 'mini-mon'
PASSWORD = 'password'

In [12]:
api_version = '2_0'
monasca_url = 'http://157.159.232.217:8070/v2.0/'
# Authenticate to Keystone
keystone_url = 'http://157.159.232.218:35357/v3/'

ks = ksclient.KSClient(auth_url=KEYSTONE_URL, username=USERNAME, password=PASSWORD)
    # construct the mon client
monasca_client = client.Client(api_version, monasca_url, token=ks.token)



# Initialize environment variables to use the Monasca CLI
%env OS_PROJECT_NAME=$PROJECT_NAME
%env OS_PASSWORD=$PASSWORD
%env OS_AUTH_URL=$KEYSTONE_URL
%env OS_USERNAME=$USERNAME
#%env OS_TEST="test"

env: OS_PROJECT_NAME=mini-mon
env: OS_PASSWORD=password
env: OS_AUTH_URL=http://157.159.232.218:35357/v3/
env: OS_USERNAME=mini-mon


In [13]:
# getting the metrics information
def get_metrics(names=[None], dimensions={}, limit=25):
    metrics = []
    for name in names:
        # Invoke the Monasca client
        metrics = metrics + monasca_client.metrics.list(name=name, dimensions=dimensions,limit=limit)
    return metrics

# function get measurements
def get_measurements(metrics, start_time = None, end_time = None, limit=None):
    measurements = []

    if start_time == None:
        start_date = datetime.datetime.utcnow() - datetime.timedelta(seconds=3600)
        start_time = start_date.strftime("%Y-%m-%dT%H:%M:%SZ")

    if end_time == None:
        end_date = datetime.datetime.utcnow() - datetime.timedelta(seconds=0)
        end_time = end_date.strftime("%Y-%m-%dT%H:%M:%SZ")

    for metric in metrics:
        # Invoke the Monasca client
        measurements.append(monasca_client.metrics.list_measurements(
                name=metric['name'],
                dimensions=metric['dimensions'],
                start_time=start_time,
                end_time=end_time))

    return measurements





def df_from_measurements2(raw, group):
    i=0
    log=0;
    df = pd.DataFrame()
    for s in raw:
        if s[0]['measurements'] and s[0]['dimensions']['hostname'] == group:
            m = np.array(s[0]['measurements'])
            timestamps = m[:, s[0]['columns'].index('timestamp')]
            df = pd.DataFrame(index = timestamps)
            break;
    m=0;
    for measure in raw:
        if measure[0]['measurements'] and measure[0]['dimensions']['hostname'] == group:
            hostname=group
            m = np.array(measure[0]['measurements'])
            timestamps = m[:, measure[0]['columns'].index('timestamp')]
            #df = pd.DataFrame(index = timestamps)
            #getting name
            if(log==1):
                print 'metric : \n'
                print measure[0]['name']
            name = measure[0]['name'];
            #getting dimensions
            if(log==1):
                print '\n Machine name : \n'
                print measure[0]['dimensions']['hostname']
            # Getting measurement
            if(log==1):
                print '\n measurements \n'
                print m#measure[0]['measurements']

            # Measurement into Array

            m = np.array(measure[0]['measurements'])


            timestamps = m[:, measure[0]['columns'].index('timestamp')]
            if(log==1):
                print timestamps


            values=m[:, measure[0]['columns'].index('value')]
            if(log==1):
                print '\n values \n '
                print values

            #vars()["df_"+str(i)] =  dict( zip( timestamps, m));

            df[name] = m[:, measure[0]['columns'].index('value')];

            if(log == 1):
                    print 'This is the dataframe'+'is'+name

            #vars()["df_"+str(i)]['hostname'] = hostname;

            if(log==1):
                print ' \n \n ***********  --------------- *********** \n'
            i=i+1;
    return df;




def Timestamp(df):
    tsp = np.array(df.index)
    i=0;
    for ind in tsp:
        tsp[i] = t.mktime(dt.datetime.strptime(repr(str(ind))[1:-1], '%Y-%m-%dT%H:%M:%S.%fZ').timetuple())
        i += 1;
    return tsp

def current_time(T):
    ts_epoch = t.mktime(dt.datetime.strptime(repr(str(T))[1:-1], '%Y-%m-%dT%H:%M:%S.%fZ').timetuple())
    ts = datetime.datetime.fromtimestamp(ts_epoch).strftime('%Y-%m-%dT%H:%M:%S.%fZ')
    
    #curt_t = strftime('%Y-%m-%dT%H:%M:%S.0Z', gmtime())
    #current_time(curt_t)
    #datetime.datetime.fromtimestamp(ts_epoch).strftime('%Y-%m-%dT%H:%M:%S.%fZ')
    return ts

# Compute a the percentage change of an array
def percentage_change(X):
    return [100.0 * a1/a2 - 100 for a1, a2 in zip(X[1:], X)]


# df in percentage
def perc_df(df):
    for i in range(len(df.columns)):
        metric_name = df.columns[i]
        df[metric_name] = np.append([1],percentage_change(df[metric_name].values))
        
        
    return df

def to_vect(X,w):
    inputs = [[X[i+e] for e in range(w)] for i in range(len(X)-w)]
    targets = [[X[j+w]] for j in range(len(X)-w)]
    
    return inputs,targets

def to_vect2(X,w):
    inputs = [[X[i+e] for e in range(w)] for i in range(len(X)-w)]
    targets = [X[j+w] for j in range(len(X)-w)]
    
    return np.array(inputs),np.array(targets)

# getting the prediction in the form of an array
def get_prediction():
    i=0;
    for inp in inputs:
        error = pow(nn.predict(inputs[i])[0]-targets[i][0],2);
        print ' \n The input is : \n\n '+str(inputs[i])+'\n\n the target is : \n\n '+str(targets[i])+'\n\n the predicted value is : \n\n '+str(nn.predict(inputs[i]))+'\n\n The error is : \n\n '+str(error)+' \n\n *** \n\n '
        i += 1;
        time.sleep(5)
        

In [20]:
look_back = 1
model = Sequential();
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_dim=look_back))
model.add(Dense(1))

In [21]:
# load YAML and create model
yaml_file = open('model.yaml', 'r')
loaded_model_yaml = yaml_file.read()
yaml_file.close()
loaded_model = model_from_yaml(loaded_model_yaml)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [22]:
T_start = "2016-09-14T07:57:26.0Z"
ks = ksclient.KSClient(auth_url=KEYSTONE_URL, username=USERNAME, password=PASSWORD)
    # construct the mon client
monasca_client = client.Client(api_version, monasca_url, token=ks.token)


i=1;

print "============== Hello  ==============="


while(True): 
    T_end = strftime('%Y-%m-%dT%H:%M:%S.0Z', gmtime()) 
    
    print " \n \n ========= Step"+str(i)+" ======== \n \n "
    
    print " \n =========  Downloading please wait  =========== \n "
    
    metrics = get_metrics(names=['load.avg_1_min','cpu.wait_perc','net.out_packets_sec','cpu.system_perc','net.in_bytes_sec','mem.free_mb']);
    measurements = get_measurements(metrics,T_start,T_end)     
    df_Ellis= df_from_measurements2(filter(None, measurements), 'ellis.jaafar.com')
    z = 0.00;
    df_Ellis['predict']= z
    df_Ellis['Timestamp']=Timestamp(df_Ellis)
    df_Ellis.index.names = [None]
    df_Ellis = df_Ellis.reset_index(drop=True)
    df_Ellis = df_Ellis.set_index('Timestamp')    
    
###    
   
    dataset1 = df_Ellis['cpu.wait_perc'].values
    dataset1 = dataset1.astype('float32')

    D1, A1 = to_vect2(dataset1,1)

    for j in range(len(df_Ellis)):
        df_Ellis['predict'][j] = (float)(model.predict(np.asanyarray([[[df_Ellis['cpu.wait_perc'].values[j]]]]),verbose=0))    

    
###   


    print "  ===================   Synchronizing  =================================  "
    
    
    df_Ellis.to_csv('Data/data_df.csv',header=True,index=True,sep=';')
       
    
    
    i += 1;
    time.sleep(10)

============== Hello  ===============
 
 
 ========= Step1 ======== 
 
 
 
 =========  Downloading please wait  =========== 
 


/home/jaafar/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:37: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



  ===================   Synchronizing  =================================  
 
 
 ========= Step2 ======== 
 
 
 
 =========  Downloading please wait  =========== 
 
  ===================   Synchronizing  =================================  
 
 
 ========= Step3 ======== 
 
 
 
 =========  Downloading please wait  =========== 
 
  ===================   Synchronizing  =================================  
 
 
 ========= Step4 ======== 
 
 
 
 =========  Downloading please wait  =========== 
 
  ===================   Synchronizing  =================================  
 
 
 ========= Step5 ======== 
 
 
 
 =========  Downloading please wait  =========== 
 
  ===================   Synchronizing  =================================  


KeyboardInterrupt: 

In [23]:
(float)(model.predict(np.asanyarray([[[df_Ellis['cpu.wait_perc'].values[0]]]]),verbose=0))    

0.0

In [24]:
df_Ellis['cpu.wait_perc'].values[0]

13.5

In [25]:
model.s

AttributeError: 'Sequential' object has no attribute 'yaml'

In [26]:
T_end = strftime('%Y-%m-%dT%H:%M:%S.0Z', gmtime())
T_end

'2016-09-14T08:40:19.0Z'

In [55]:
#T is a string 
def current_time(T,h):
    if T:
        ts = t.mktime(dt.datetime.strptime(repr(str(T))[1:-1], '%Y-%m-%dT%H:%M:%S.%fZ').timetuple())
    #ts = datetime.datetime.fromtimestamp(ts_epoch).strftime('%Y-%m-%dT%H:%M:%S.%fZ')
    
    if T is None:
        ts_epoch = strftime('%Y-%m-%dT%H:%M:%S.0Z', gmtime())
        ts = t.mktime(dt.datetime.strptime(repr(str(ts_epoch))[1:-1], '%Y-%m-%dT%H:%M:%S.%fZ').timetuple())
        ts = ts - 60*60*h;
        s =  datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%dT%H:%M:%S.%fZ')
        
        
        
    return s



In [58]:

current_time(T=None,h=0.5)

'2016-09-14T08:26:22.000000Z'